# Machine Learning Pipelines

## Machine Learning Orchestration with Amazon SageMaker Pipelines

In [5]:
import os
import sagemaker
import logging
import boto3
import time
import pandas as pd
import json
import botocore
from botocore.exceptions import ClientError


# ========================== low-level service client of the boto3 session ==========================
config = botocore.config.Config(user_agent_extra='bedissj-1699438736259')


sm = boto3.client(service_name='sagemaker', 
                  config=config)

sm_runtime = boto3.client('sagemaker-runtime',
                          config=config)

sess = sagemaker.Session(sagemaker_client=sm,
                         sagemaker_runtime_client=sm_runtime)

bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = sess.boto_region_name


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


![model_registry](./img/model_registry.png)

### 1.1 Configure Data Processing Step

In [ ]:
raw_data_s3_uri = f's3://{bucket}/data/transformed_querying/month1'

In [8]:
from sagemaker.sklearn.processing import SKLearnProcessor


sklearn_processor = SKLearnProcessor(
    framework_version=FRAMEWORK_VERSION,
    role=role,
    instance_type=processing_instance_type,
    instance_count = processing_instance_count,
    env={'AWS_DEFAULT_REGION': region}
)


Init signature:
SKLearnProcessor(
    framework_version: str,
    role: Union[str, sagemaker.workflow.entities.PipelineVariable, NoneType] = None,
    instance_count: Union[int, sagemaker.workflow.entities.PipelineVariable] = None,
    instance_type: Union[str, sagemaker.workflow.entities.PipelineVariable] = None,
    command: Optional[List[str]] = None,
    volume_size_in_gb: Union[int, sagemaker.workflow.entities.PipelineVariable] = 30,
    volume_kms_key: Union[str, sagemaker.workflow.entities.PipelineVariable, NoneType] = None,
    output_kms_key: Union[str, sagemaker.workflow.entities.PipelineVariable, NoneType] = None,
    max_runtime_in_seconds: Union[int, sagemaker.workflow.entities.PipelineVariable, NoneType] = None,
    base_job_name: Optional[str] = None,
    sagemaker_session: Optional[sagemaker.session.Session] = None,
    env: Optional[Dict[str, Union[str, sagemaker.workflow.entities.PipelineVariable]]] = None,
    tags: Union[List[Dict[str, Union[str, sagemaker.workflow.

In [6]:
from sagemaker.workflow.steps import ProcessingStep


processing_step = ProcessingStep(
    name='data-processing',
    code='./src/processing.py'
    processor=sklearn_processor,
    inputs=processing_inputs,
    outputs=processing_outputs,
    job_arguments=processing_arguments,
)

Init signature:
ProcessingStep(
    name: str,
    step_args: sagemaker.workflow.pipeline_context._JobStepArguments = None,
    processor: sagemaker.processing.Processor = None,
    display_name: str = None,
    description: str = None,
    inputs: List[sagemaker.processing.ProcessingInput] = None,
    outputs: List[sagemaker.processing.ProcessingOutput] = None,
    job_arguments: List[str] = None,
    code: str = None,
    property_files: List[sagemaker.workflow.properties.PropertyFile] = None,
    cache_config: sagemaker.workflow.steps.CacheConfig = None,
    depends_on: Optional[List[Union[str, sagemaker.workflow.steps.Step, ForwardRef('StepCollection')]]] = None,
    retry_policies: List[sagemaker.workflow.retry.RetryPolicy] = None,
    kms_key=None,
)
Docstring:      `ProcessingStep` for SageMaker Pipelines Workflows.
Init docstring:
Construct a `ProcessingStep`, given a `Processor` instance.

In addition to the `Processor` instance, the other arguments are those that are supplied